In [1]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten
from keras.layers import Dropout
from keras.utils import to_categorical

!pip install scikit-plot
!pip install ucimlrepo

In [2]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
cdc_diabetes_health_indicators = fetch_ucirepo(id=891)

# data (as pandas dataframes)
X = cdc_diabetes_health_indicators.data.features
y = cdc_diabetes_health_indicators.data.targets

In [4]:
X_split = np.array_split(X, 4)
y_split = np.array_split(y, 4)
trainX_a, testX_a, trainY_a, testY_a = train_test_split(X_split[0], y_split[0], test_size=0.2)

In [5]:
model_a = Sequential()
model_a.add(Dense(3, activation='relu', input_shape=(trainX_a.shape[1], 1)))
model_a.add(Dropout(0.1))
model_a.add(Flatten())
model_a.add(Dense(1, activation='sigmoid'))
model_a.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_a.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 21, 3)             6         
                                                                 
 dropout (Dropout)           (None, 21, 3)             0         
                                                                 
 flatten (Flatten)           (None, 63)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 64        
                                                                 
Total params: 70 (280.00 Byte)
Trainable params: 70 (280.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
history_a = model_a.fit(trainX_a, trainY_a, epochs=3)

Epoch 1/3
1586/1586 [==============================] - 4s 2ms/step - loss: 0.4471 - accuracy: 0.8452
Epoch 2/3
1586/1586 [==============================] - 4s 3ms/step - loss: 0.3475 - accuracy: 0.8620
Epoch 3/3
1586/1586 [==============================] - 3s 2ms/step - loss: 0.3363 - accuracy: 0.8644


In [7]:
score_a = model_a.evaluate(testX_a, testY_a, verbose=0)
print(score_a)

[0.3296794295310974, 0.8632923364639282]


In [8]:
pesiTmp_a = model_a.get_weights()

In [10]:
import requests as r
import json

pesiTmp2_a = [pesiTmp_a[0][0], pesiTmp_a[1]]
# Conversione dei pesi per renderli compatibili con la codifica JSON e del server
pesi_a = '{\"l1\" : [[' + np.array2string(pesiTmp_a[0][0][0]) + '],[' + np.array2string(pesiTmp_a[0][0][1]) + '],[' + np.array2string(pesiTmp_a[0][0][2]) + ']], \"a\" : \"' + np.array2string(pesiTmp_a[1]) + '\"}'
pesi_json = json.loads(pesi_a)
print(pesi_a)

{"l1" : [[0.23176591],[-0.05129073],[0.65986884]], "a" : "[-0.00655607  0.42114455 -0.00529602]"}


In [11]:
url = 'http://93.65.162.11:3001'
x = r.post(url + '/nodo_a', data = pesi_a, headers={"Content-Type": "application/json"})
print(x)

<Response [200]>


In [ ]:
media = r.get(url + '/pesi')
dict = json.loads(media.text)

pesi_tmp = np.array(dict['l1'])
pesi_tmp_a = np.array(dict['a'])

pesi_nodi = pesi_tmp.reshape((1,3))
pesi_archi = pesi_tmp_a.reshape((1,3))
pesi_archi[pesi_archi == None] = 0

In [ ]:
pesi_tmp_a = model_a.get_weights()

pesi_tmp_a[0][0][0] = pesi_nodi[0][0]
pesi_tmp_a[0][0][1] = pesi_nodi[0][1]
pesi_tmp_a[0][0][2] = pesi_nodi[0][2]

pesi_tmp_a[1][0] = pesi_archi[0][0]
pesi_tmp_a[1][1] = pesi_archi[0][1]
pesi_tmp_a[1][2] = pesi_archi[0][2]

In [ ]:
model_a.set_weights(pesi_tmp_a)
history_a = model_a.fit(trainX_a, trainY_a, epochs=3)
score_a = model_a.evaluate(testX_a, testY_a, verbose=0)